# Can building-integrated PV, coupled with ice storage, cool a house at night?

TODO:

- make an accurate model of roof profiles, instead of using a flat GHI average

- daytime cooling - can we also cool the house during midday and evening?

- can we power other appliances during the day, while still having enough energy to cool the house?

- 

In [ ]:
import pvlib
import json
import math
import os
from pprint import pprint
from datetime import datetime
from pytz import timezone
import statistics

import pandas as pd

In [ ]:
enthalpy_of_fusion_ice_j_g = 333.55

specific_heat_capacity_water_j_g = 4.184
specific_heat_capacity_concrete_j_g = 0.880
specific_heat_capacity_wood_j_g = 1.7
specific_heat_capacity_air_j_g = 1.000

concrete_density_kg_m3 = 2400
wood_density_kg_m3 = 700
air_density_room_temperature_kg_m3 = 1.2

room_temperature = 24
outside_temperature = 34

# heat inflow
# from outside to inside
# cooling the room itself

In [ ]:
latent_heat_per_kg_ice_kwh = enthalpy_of_fusion_ice_j_g * 1000 / (3600 * 1000)

print("Latent Energy in 1kg of ice(kWh): ", round(latent_heat_per_kg_ice_kwh, 4))

print("Latent Energy in 1 ton of ice(kWh): ", round(latent_heat_per_kg_ice_kwh * 1000, 4))

## Estimating Refrigeration: Heat Transfer Method

### Heat Source

conduction through walls
human thermoregulation

https://www.engineeringtoolbox.com/air-change-rate-room-d_867.html


### Heat buffer

walls
furniture
human

### Heat Sink

air conditioner




In [ ]:
# how much energy does it take to cool a room down from 34C to 24C?
# -- this is the lower bound of the energy actually needed


medium_bedroom_volume_air_m3 = 12 * 3
air_change_rate_n_h = 15
num_hours_sleep = 8
medium_bedroom_mass_air_kg = medium_bedroom_volume_air_m3 * air_density_room_temperature_kg_m3
medium_bedroom_heat_capacity_kwh_C = medium_bedroom_mass_air_kg * 1000 * specific_heat_capacity_air_j_g / 3600000
medium_bedroom_removed_energy_kwh = (
    (outside_temperature - room_temperature) 
    * air_change_rate_n_h 
    * num_hours_sleep 
    * medium_bedroom_heat_capacity_kwh_C
)

# add conductive heat transfer through the walls
conduction_area = (2 * 4 * 3) + (2 * 3 * 3)
conduction_heat_power_w = (outside_temperature - room_temperature) * conduction_area * 1.28
conduction_removed_energy_kwh = conduction_heat_power_w * num_hours_sleep / 1000

# add thermal mass of walls - assuming 5cm on each side
room_walls_volume_m3 = (4) * (3 + 0.1) * (3 + 0.1) - 12 * 3
medium_bedroom_mass_concrete_kg = room_walls_volume_m3 * concrete_density_kg_m3
medium_bedroom_concrete_heat_capacity_kwh_C = medium_bedroom_mass_concrete_kg * 1000 * specific_heat_capacity_concrete_j_g / 3600000
medium_bedroom_concrete_removed_energy_kwh = (
    (outside_temperature - room_temperature) 
    * medium_bedroom_concrete_heat_capacity_kwh_C
)

# add thermal mass of furniture
medium_bedroom_mass_wood_kg = 150
medium_bedroom_wood_heat_capacity_kwh_C = medium_bedroom_mass_wood_kg * 1000 * specific_heat_capacity_wood_j_g / 3600000
medium_bedroom_wood_removed_energy_kwh = (
    (outside_temperature - room_temperature) 
    * medium_bedroom_wood_heat_capacity_kwh_C
)

two_people_heat_power_w = 60 * 2
two_people_removed_energy_kwh = two_people_heat_power_w * num_hours_sleep / 1000

medium_bedroom_removed_energy_ice_kg = (medium_bedroom_removed_energy_kwh * 3600000 / enthalpy_of_fusion_ice_j_g) / 1000
two_people_removed_energy_ice_kg = (two_people_removed_energy_kwh * 3600000 / enthalpy_of_fusion_ice_j_g) / 1000
medium_bedroom_concrete_removed_energy_ice_kg = (medium_bedroom_concrete_removed_energy_kwh * 3600000 / enthalpy_of_fusion_ice_j_g) / 1000
furniture_removed_energy_ice_kg = (medium_bedroom_wood_removed_energy_kwh * 3600000 / enthalpy_of_fusion_ice_j_g) / 1000
conduction_removed_energy_ice_kg = (conduction_removed_energy_kwh * 3600000 / enthalpy_of_fusion_ice_j_g) / 1000

print("Air Removed Energy (kgs of ice): ", medium_bedroom_removed_energy_ice_kg)
print("Human Metabolism Removed Energy (kgs of ice): ", two_people_removed_energy_ice_kg)
print("Concrete Walls removed energy (kgs of ice): ", medium_bedroom_concrete_removed_energy_ice_kg)
print("Furniture removed energy (kgs of ice): ", furniture_removed_energy_ice_kg)
print("Conduction removed energy (kgs of ice): ", conduction_removed_energy_ice_kg)

print()

total_removed_energy_kwh = (
    medium_bedroom_removed_energy_kwh 
    + two_people_removed_energy_kwh 
    + medium_bedroom_concrete_removed_energy_kwh
    + medium_bedroom_wood_removed_energy_kwh
    + conduction_removed_energy_kwh
)

total_removed_energy_ice_kg = (
    medium_bedroom_removed_energy_ice_kg 
    + two_people_removed_energy_ice_kg 
    + medium_bedroom_concrete_removed_energy_ice_kg
    + furniture_removed_energy_ice_kg
    + conduction_removed_energy_ice_kg
)

print(
    "Min. Energy needed to cool down a medium bedroom(kWh): ", total_removed_energy_kwh
)
print(
    "Min. Energy needed to cool down a medium bedroom(kgs of ice): ", total_removed_energy_ice_kg
)

## Estimating Refrigeration: Heuristic Method


1 ton of refrigeration

= rate of heat transfer that results in the freezing or melting of 1 short ton (2,000 lb; 907 kg) of pure ice at 0 °C (32 °F) in 24 hours

= 12000 BTU/h

= 3.516853 kW

In [ ]:
def get_tmy_from_nsrdb(lat, lon, timezone, api_key, full_name, email, affiliation, data_source='himawari'):

    if data_source == "himawari":
        url = "http://developer.nrel.gov/api/nsrdb/v2/solar/himawari-tmy-download.csv"
    elif data_source == "goes":
        url = "http://developer.nrel.gov/api/nsrdb/v2/solar/nsrdb-GOES-tmy-v4-0-0-download.csv"
    elif data_source == "meteosat":
        url = "http://developer.nrel.gov/api/nsrdb/v2/solar/nsrdb-msg-v1-0-0-tmy-download.csv"
    else:
        raise ValueError(f"Invalid data source {data_source}")

    key = f'{data_source}_tmy'
    if os.path.exists(key) is False:
        os.mkdir(key)

    filename = f"pvlib_experiments/{key}/{data_source}_tmy_{lat}_{lon}.csv"
    metadata_filename = f"pvlib_experiments/{key}/{data_source}_tmy_{lat}_{lon}.json"

    if os.path.exists(filename) is False:
        df, metadata = pvlib.iotools.get_nsrdb_psm4_tmy(
            lat,
            lon,
            api_key,
            email,
            year='tmy',
            time_step=60,
            leap_day=False,
            full_name=full_name,
            affiliation=affiliation,
            utc=True,
            map_variables=True,
            url=url,
            timeout=30
        )
    
        df.to_csv(filename)
        with open(metadata_filename, mode='w') as f:
            json.dump(metadata, f)

    df = pd.read_csv(filename, index_col=0)
    with open(metadata_filename, mode='r') as f:
        metadata = json.load(f)

    df["Year"] = 1990
    df["sitetime"] = df.apply(lambda x: datetime(int(x["Year"]), int(x["Month"]), int(x["Day"]), int(x["Hour"]), int(x["Minute"])), axis=1)
    df.index = pd.DatetimeIndex(df["sitetime"], tz="UTC").tz_convert(timezone)


    df = df[['temp_air', 'temp_dew', 'dhi', 'dni', 'ghi', 'albedo', 'pressure', 'wind_direction', 'wind_speed'
    ]]
    
    return df, metadata

In [ ]:
api_key = '9nbF8PWcvcjXeQALyMXmOfmefnOTb942xxHGvR0M'
full_name = "David Pratama Widjaja"
affiliation = ""
email = "widjaja.david@ymail.com"
reason = 'Research into Indonesian and ASEAN energy transition'
join_mailing_list = 'true'

jakarta_lat = -6.175389
jakarta_lon = 106.827139
jakarta_data_source = 'himawari'
jakarta_tz = timezone('Asia/Jakarta')

amsterdam_lat = 52.372778
amsterdam_lon = 4.893611
amsterdam_data_source = 'meteosat'
amsterdam_tz = timezone('Europe/Amsterdam')

surabaya_lat = -7.245833
surabaya_lon = 112.737778
surabaya_data_source = 'himawari'
surabaya_tz = timezone('Asia/Makassar')

vancouver_lat = 49.260833
vancouver_lon = -123.113889
vancouver_data_source = 'goes'
vancouver_tz = timezone('America/Vancouver')

toronto_lat = 43.741667
toronto_lon = -79.373333
toronto_data_source = 'goes'
toronto_tz = timezone('America/Toronto')

In [ ]:

# 1 ton of refrigeration
# rate of heat transfer that results in the freezing or melting of 1 short ton (2,000 lb; 907 kg) of pure ice at 0 °C (32 °F) in 24 hours
# = 12000 BTU/h
# = 3.516853 kW

tons_of_refrigeration_to_btu_per_h = 12000
tons_of_refrigeration_to_kw = 3.516853
SEER = 20
# very conservative!
SEER_freezer = 10
tons_of_refrigration_energy_to_kwh = 12000 * 1055 / 3600 / 1000

# tons of refrigeration (power) needed to cool a room
tons_of_refrigeration_per_24_h_power = {
    "Small Bedroom": 0.5,
    "Medium Bedroom": 1.0,
    "Large Bedroom": 1.5,
    "Study": 1.0,
    "Living Room": 1.0,
}

tons_of_refrigeration_per_h_power = {k: v for k, v in tons_of_refrigeration_per_24_h_power.items()}

tons_of_refrigeration_per_8h_energy = {k: v * 8 for k, v in tons_of_refrigeration_per_h_power.items()}

# how many actual tons of ice is needed to produce "1 ton of refrigeration" for 8 hours
actual_tons_of_refrigeration_per_8h_energy = {k: v / 24 * 8 for k, v in tons_of_refrigeration_per_h_power.items()}
actual_kg_of_ice_per_8h_energy = {k: v * 907 for k, v in actual_tons_of_refrigeration_per_8h_energy.items()}

kwh_per_8h_energy_cooling = {k: v * tons_of_refrigration_energy_to_kwh for k, v in tons_of_refrigeration_per_8h_energy.items()}
kwh_per_8h_energy_cooling_2 = {k: v * enthalpy_of_fusion_ice_j_g * 907 * 1000 / 3600000 for k, v in actual_tons_of_refrigeration_per_8h_energy.items()}

kwh_per_8h_energy_electricity_aircon = {k: v * 12000 / SEER / 1000 for k, v in tons_of_refrigeration_per_8h_energy.items()}
kwh_per_8h_energy_electricity_freezer = {k: v * 12000 / SEER_freezer / 1000 for k, v in tons_of_refrigeration_per_8h_energy.items()}


cooling_layouts_day = {
    "Working Class House": [],
    "Lower Middle Class House": ["Living Room"],
    "Middle Class House": ["Living Room"],
    "Upper Middle Class House": ["Living Room", "Study"],
    "Upper Class House": ["Living Room", "Living Room", "Study"]
}

cooling_layouts_night = {
    "Working Class House": ["Small Bedroom"],
    "Lower Middle Class House": ["Small Bedroom", "Small Bedroom"],
    "Middle Class House": ["Small Bedroom", "Medium Bedroom"],
    "Upper Middle Class House": ["Small Bedroom", "Small Bedroom", "Medium Bedroom"],
    "Upper Class House": ["Small Bedroom", "Small Bedroom", "Medium Bedroom", "Large Bedroom"],
}

# Working Class House 20m2 - 40m2  LT 20
# Lower Middle Class House is 40m2 - 85m2 LT 40
# Middle Class House is 85m2 - 130m2 LT 80
# Upper Middle Class House is 130m2 - 200m2 LT 160
# Upper Class House is 200m2 - 1000m2 LT 400

# Studio 20m2 - 50m2

# 1 small bedroom at night is 6m2
# 1 medium sized bedroom at night is 12m2
# 1 large bedroom at night is 25m2

# area of the house
layout_areas_m2 = {
    "Working Class House": 30,
    "Lower Middle Class House": 85,
    "Middle Class House": 130,
    "Upper Middle Class House": 200,
    "Upper Class House": 600
}

building_areas_m2 = {
    "Working Class House": 30,
    "Lower Middle Class House": 75,
    "Middle Class House": 100,
    "Upper Middle Class House": 140,
    "Upper Class House": 300
}

# area of the entire property
land_areas_m2 = {
    "Working Class House": 30,
    "Lower Middle Class House": 90,
    "Middle Class House": 120,
    "Upper Middle Class House": 160,
    "Upper Class House": 400
}

# let the usable solar area be a fraction of the building area
USABLE_ROOF_PERCENTAGE = 0.6
usable_solar_area_m2 = {k: v * USABLE_ROOF_PERCENTAGE for k, v in building_areas_m2.items()}

num_modules = {k: v / 1.7 for k, v in usable_solar_area_m2.items()}


In [ ]:
# get the kwh requirements of each housing type

actual_kg_of_ice_per_house = {}
actual_kwh_cooling_per_house = {}
kwh_freezer_electricity_per_house = {}

for housing_type, rooms_list in cooling_layouts_night.items():

    actual_kg_of_ice_per_house[housing_type] = sum([actual_kg_of_ice_per_8h_energy[r] for r in rooms_list])
    actual_kwh_cooling_per_house[housing_type] = sum([kwh_per_8h_energy_cooling[r] for r in rooms_list])
    kwh_freezer_electricity_per_house[housing_type] = sum([kwh_per_8h_energy_electricity_freezer[r] for r in rooms_list])

In [ ]:
actual_kwh_cooling_per_house

In [ ]:
actual_kg_of_ice_per_house

In [ ]:
kwh_freezer_electricity_per_house

In [ ]:
kwh_per_8h_energy_cooling

In [ ]:
kwh_per_8h_energy_cooling_2

In [ ]:
land_areas_m2

In [ ]:
usable_solar_area_m2

In [ ]:
num_modules

In [ ]:
num_modules

In [ ]:
df, metadata = get_tmy_from_nsrdb(jakarta_lat, jakarta_lon, jakarta_tz, api_key, full_name, email, affiliation, data_source='himawari')

In [ ]:
# get the irradiance per day in W/m2, and sum it to kwh/m2 per day

MODULE_EFFICIENCY = 0.20
ROOF_ANGLE = 30

roof_area_m2 = {k: v / math.cos(ROOF_ANGLE * math.pi / 180) for k, v in building_areas_m2.items()}
df, metadata = get_tmy_from_nsrdb(jakarta_lat, jakarta_lon, jakarta_tz, api_key, full_name, email, affiliation, data_source='himawari')
df.index = pd.DatetimeIndex(pd.date_range(datetime(2023, 1, 1), datetime(2024, 1, 1), inclusive='left', freq='1h', tz='UTC').tz_convert(jakarta_tz))

times = pd.date_range(datetime(2023, 1, 1), datetime(2024, 1, 1), inclusive='left', freq='1h', tz='UTC').tz_convert(jakarta_tz)
solarposition_df = pvlib.location.Location(jakarta_lat, jakarta_lon).get_solarposition(times)

daily_kwh_electricity_per_m2 = (df["ghi"].resample('1D').sum() * 3600 * MODULE_EFFICIENCY/ 3600000)[:-1]
monthly_kwh_electricity_per_m2 = (df["ghi"].resample('1MS').sum() * 3600 * MODULE_EFFICIENCY/ 3600000)[:-1]

# now calculate POA
surface_tilt = 30
surface_azimuths = [0, 90, 180, 270]
solar_zenith = solarposition_df['apparent_zenith']
solar_azimuth = solarposition_df['azimuth']
dni = df['dni']
ghi = df['ghi']
dhi = df['dhi']

# for each of the different azimuth orientations, get the poa W/m2 for each timestamp
# then, convert to kwh/h/m2 for each timestamp
# finally, convert to kwh/h for each roof slope


kwh_m2_daily_list = []
kwh_m2_monthly_list = []
for surface_azimuth in surface_azimuths:
    total_irradiance_df = pvlib.irradiance.get_total_irradiance(surface_tilt, surface_azimuth, solar_zenith, solar_azimuth, dni, ghi, dhi)
    daily_kwh_m2 = total_irradiance_df['poa_global'].resample('1D').sum().rename(f'poa_{surface_azimuth}') * MODULE_EFFICIENCY / 1000
    monthly_kwh_m2 = total_irradiance_df['poa_global'].resample('1MS').sum().rename(f'poa_{surface_azimuth}') * MODULE_EFFICIENCY / 1000

    kwh_m2_daily_list.append(daily_kwh_m2)
    kwh_m2_monthly_list.append(monthly_kwh_m2)

daily_kwh_electricity_per_m2 = pd.concat(kwh_m2_daily_list, axis=1).mean(axis=1).rename('kwh_per_m2_average')
monthly_kwh_electricity_per_m2 = pd.concat(kwh_m2_monthly_list, axis=1).mean(axis=1).rename('kwh_per_m2_average')

# for every housing type, create a monthly time series of electricity produced
daily_cooling_sufficiency_ratio = pd.DataFrame(
    {f"{k}": v * daily_kwh_electricity_per_m2 / (kwh_freezer_electricity_per_house[k] * 1.5) for k, v in usable_solar_area_m2.items()}
)
monthly_cooling_sufficiency_ratio = pd.DataFrame(
    {f"{k}": v * monthly_kwh_electricity_per_m2 / (30 * kwh_freezer_electricity_per_house[k] * 1.5) for k, v in usable_solar_area_m2.items()}
)

daily_excess_electricity_kwh = pd.DataFrame(
    {f"{k}": v * daily_kwh_electricity_per_m2 - kwh_freezer_electricity_per_house[k] * 1.5 - 10 for k, v in usable_solar_area_m2.items()}
)
monthly_excess_electricity_kwh = pd.DataFrame(
    {f"{k}": v * monthly_kwh_electricity_per_m2 - 30 * kwh_freezer_electricity_per_house[k] * 1.5 - 10 * 30 for k, v in usable_solar_area_m2.items()}
)

daily_electricity_kwh = pd.DataFrame(
    {f"{k}": v * daily_kwh_electricity_per_m2 for k, v in usable_solar_area_m2.items()}
)
monthly_electricity_kwh = pd.DataFrame(
    {f"{k}": v * monthly_kwh_electricity_per_m2 for k, v in usable_solar_area_m2.items()}
)



In [ ]:
daily_electricity_kwh.plot()

In [ ]:
monthly_cooling_sufficiency_ratio.plot.bar()

In [ ]:
daily_cooling_sufficiency_ratio.plot()

In [ ]:
monthly_excess_electricity_kwh[:-1].plot.bar()

In [ ]:
daily_excess_electricity_kwh["Working Class House"].rolling('3D').mean().plot(ylim=[0, 50])